# 10.1 GroupBy机制

In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                  'key2':['one','two','one','two','one'],
                  'data1':np.random.randn(5),
                  'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.666490,-0.982017
1,a,two,-0.092814,0.991393
2,b,one,0.595810,0.062268
3,b,two,-0.373024,0.350002
4,a,one,0.839849,0.619818


In [3]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                  'key2':['one','two','one','two','one'],
                  'data1':np.random.randn(5),
                  'data2':np.random.randn(5)})
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    0.118881
b   -1.508514
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one    -0.113187
      two     0.583017
b     one    -1.433102
      two    -1.583925
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,-0.113187,0.583017
b,-1.433102,-1.583925


In [13]:
states = np.array(['Ohio','California','California','California','Ohio'])
years = np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005   -0.500454
            2006   -1.433102
Ohio        2005   -0.967028
            2006    0.740654
Name: data1, dtype: float64

In [14]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.118881,0.045733
b,-1.508514,-0.854688


In [15]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.113187  0.323490
     two   0.583017 -0.509781
b    one  -1.433102 -0.901971
     two  -1.583925 -0.807406

In [16]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

# 对分组进行迭代

In [17]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.967028 -0.097681
1    a  two  0.583017 -0.509781
4    a  one  0.740654  0.744661
b
  key1 key2     data1     data2
2    b  one -1.433102 -0.901971
3    b  two -1.583925 -0.807406


In [18]:
for(k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.967028 -0.097681
4    a  one  0.740654  0.744661
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.583017 -0.509781
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.433102 -0.901971
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.583925 -0.807406


In [19]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.433102,-0.901971
3,b,two,-1.583925,-0.807406


In [20]:
#groupby默认是在axis=0上进行分组的，通过设置也可以在其他任何轴上进行分组
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [23]:
grouped = df.groupby(df.dtypes,axis=1)
for dtype,group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.967028 -0.097681
1  0.583017 -0.509781
2 -1.433102 -0.901971
3 -1.583925 -0.807406
4  0.740654  0.744661
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


# 选取一列或列的子集

In [ ]:
#对于由DataFrame产生的GroupBy对象，如果用一个或一组列名对其进行索引，就能实现选取部分列进行聚合的目的
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
#是以下代码的语法糖：
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [24]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.323490
     two  -0.509781
b    one  -0.901971
     two  -0.807406

In [25]:
s_grouped = df.groupby(['key1','key2'])['data2']
s_grouped

In [26]:
s_grouped.mean()

key1  key2
a     one     0.323490
      two    -0.509781
b     one    -0.901971
      two    -0.807406
Name: data2, dtype: float64

# 通过字典或Series进行分组

In [28]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns=['a','b','c','d','e'],
                     index=['Joe','Steve','Wes','Jim','Travis'])
people

,a,b,c,d,e
Joe,-0.047321,0.442620,0.276852,-0.838153,1.486938
Steve,1.119226,-0.905800,-0.155690,-0.504585,1.351122
Wes,0.977421,-0.790983,-1.027854,-1.241442,-0.906511
Jim,-0.933995,-1.194195,-0.534830,1.293501,0.887170
Travis,1.226412,-0.315565,1.052792,0.120648,0.775665


In [32]:
mapping = {'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}
by_column = people.groupby(mapping,axis=1)
by_column.sum()

,blue,red
Joe,-0.561302,1.882237
Steve,-0.660275,1.564548
Wes,-2.269296,-0.720072
Jim,0.758671,-1.241020
Travis,1.173440,1.686512


In [33]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [34]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,2,3
Jim,2,3
Travis,2,3


# 通过函数进行分组

In [35]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.003894,-1.542558,-1.285832,-0.786095,1.467597
5,1.119226,-0.905800,-0.155690,-0.504585,1.351122
6,1.226412,-0.315565,1.052792,0.120648,0.775665


In [36]:
key_list = ['one','one','one','two','two']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -0.047321 -0.790983 -1.027854 -1.241442 -0.906511
  two -0.933995 -1.194195 -0.534830  1.293501  0.887170
5 one  1.119226 -0.905800 -0.155690 -0.504585  1.351122
6 two  1.226412 -0.315565  1.052792  0.120648  0.775665

# 根据索引级别分组

In [38]:
#层次化索引数据集最方便的地方在于它能够根据轴索引的一个级别进行聚合
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],
                                    names=['city','tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5),columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      1.533005 -0.580511  0.269482 -0.528734  0.384968
1      0.086428 -0.453582  1.542072 -0.355804 -0.263017
2     -0.298740  0.487676 -1.632587  0.120652  0.629945
3      0.534599 -0.859120 -0.372478  0.928781 -0.224482

In [40]:
hier_df.groupby(level='city',axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 10.2 数据聚合

In [ ]:
#聚合指的是任何能够从数组产生标量的数据转换过程

In [41]:
df

,key1,key2,data1,data2
0,a,one,-0.967028,-0.097681
1,a,two,0.583017,-0.509781
2,b,one,-1.433102,-0.901971
3,b,two,-1.583925,-0.807406
4,a,one,0.740654,0.744661


In [42]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.709127
b   -1.448185
Name: data1, dtype: float64

In [43]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.707683,1.254442
b,0.150823,0.094565


In [44]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.118881  0.943722 -0.967028 -0.192006  0.583017  0.661836   
b      2.0 -1.508514  0.106648 -1.583925 -1.546219 -1.508514 -1.470808   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.740654   3.0  0.045733  0.639400 -0.509781 -0.303731 -0.097681   
b    -1.433102   2.0 -0.854688  0.066868 -0.901971 -0.878330 -0.854688   

                          
           75%       max  
key1                      
a     0.323490  0.744661  
b    -0.831047 -0.807406

# 面向列的多函数应用

In [5]:
tips = pd.read_csv('C:\\Mine\\Github\\Python\\tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [6]:
grouped = tips.groupby(['day','smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [9]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [11]:
import numpy as np
grouped_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [13]:
functions = ['count','mean','max']
result = grouped['tip_pct','total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [14]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [15]:
ftuples = [('Durchschnitt','mean'),('Abweichung',np.v)]
grouped['tip_pct','total_bill'].agg(ftuples)

ValueError: no results

In [16]:
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [17]:
grouped.agg({'tip_pct':['min','max','mean','std'],
            'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

# 以“没有行索引”的形式返回聚合数据

In [18]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 10.3 apply：一般性的“拆分-应用-合并”

In [19]:
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [20]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [22]:
tips.groupby(['smoker','day']).apply(top,n=1,column=['total_bill','tip','smoker','day','time','size'])

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [24]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [25]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [26]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill        tip       size    tip_pct
day  smoker                                                   
Fri  No     count    4.000000   4.000000   4.000000   4.000000
            mean    18.420000   2.812500   2.250000   0.151650
            std      5.059282   0.898494   0.500000   0.028123
            min     12.460000   1.500000   2.000000   0.120385
            25%     15.100000   2.625000   2.000000   0.137239
            50%     19.235000   3.125000   2.000000   0.149241
            75%     22.555000   3.312500   2.250000   0.163652
            max     22.750000   3.500000   3.000000   0.187735
     Yes    count   15.000000  15.000000  15.000000  15.000000
            mean    16.813333   2.714000   2.066667   0.174783
            std      9.086388   1.077668   0.593617   0.051293
            min      5.750000   1.000000   1.000000   0.103555
            25%     11.690000   1.960000   2.000000   0.133739
            50%     13.420000   2.500000   2.000000   0.173913
            75%     18.665000   3.240000   2.000000   0.209240
            max     40.170000   4.730000   4.000000   0.263480
Sat  No     count   45.000000  45.000000  45.000000  45.000000
            mean    19.661778   3.102889   2.555556   0.158048
            std      8.939181   1.642088   0.784960   0.039767
            min      7.250000   1.000000   1.000000   0.056797
            25%     14.730000   2.010000   2.000000   0.136240
            50%     17.820000   2.750000   2.000000   0.150152
            75%     20.650000   3.390000   3.000000   0.183915
            max     48.330000   9.000000   4.000000   0.291990
     Yes    count   42.000000  42.000000  42.000000  42.000000
            mean    21.276667   2.875476   2.476190   0.147906
            std     10.069138   1.630580   0.862161   0.061375
            min      3.070000   1.000000   1.000000   0.035638
            25%     13.405000   2.000000   2.000000   0.091797
            50%     20.390000   2.690000   2.000000   0.153624
...                       ...        ...        ...        ...
Sun  No     std      8.130189   1.224785   1.032674   0.042347
            min      8.770000   1.010000   2.000000   0.059447
            25%     14.780000   2.000000   2.000000   0.139780
            50%     18.430000   3.020000   3.000000   0.161665
            75%     25.000000   3.920000   4.000000   0.185185
            max     48.170000   6.000000   6.000000   0.252672
     Yes    count   19.000000  19.000000  19.000000  19.000000
            mean    24.120000   3.516842   2.578947   0.187250
            std     10.442511   1.261151   0.901591   0.154134
            min      7.250000   1.500000   2.000000   0.065660
            25%     17.165000   3.000000   2.000000   0.097723
            50%     23.100000   3.500000   2.000000   0.138122
            75%     32.375000   4.000000   3.000000   0.215325
            max     45.350000   6.500000   5.000000   0.710345
Thur No     count   45.000000  45.000000  45.000000  45.000000
            mean    17.113111   2.673778   2.488889   0.160298
            std      7.721728   1.282964   1.179796   0.038774
            min      7.510000   1.250000   1.000000   0.072961
            25%     11.690000   1.800000   2.000000   0.137741
            50%     15.950000   2.180000   2.000000   0.153492
            75%     20.270000   3.000000   2.000000   0.184843
            max     41.190000   6.700000   6.000000   0.266312
     Yes    count   17.000000  17.000000  17.000000  17.000000
            mean    19.190588   3.030000   2.352941   0.163863
            std      8.355149   1.113491   0.701888   0.039389
            min     10.340000   2.000000   2.000000   0.090014
            25%     13.510000   2.000000   2.000000   0.148038
            50%     16.470000   2.560000   2.000000   0.153846
            75%     19.810000   4.000000   2.000000   0.194837
            max     43.110000   5.000000   4.000000   0.241255

[64 rows x 4 columns]

# 禁止分组键

In [27]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


# 分位数和桶分析

In [28]:
frame = pd.DataFrame({'data1':np.random.randn(1000),
                     'data2':np.random.randn(1000)})
quartiles = pd.cut(frame.data1,4)
quartiles[:10]

0      (0.193, 1.648]
1     (-1.261, 0.193]
2      (0.193, 1.648]
3     (-1.261, 0.193]
4     (-1.261, 0.193]
5      (1.648, 3.103]
6      (0.193, 1.648]
7      (0.193, 1.648]
8    (-2.722, -1.261]
9      (0.193, 1.648]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.722, -1.261] < (-1.261, 0.193] < (0.193, 1.648] < (1.648, 3.103]]

In [29]:
def get_stats(group):
    return {'min':group.min(),'max':group.max(),
           'count':group.count(),'mean':group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.722, -1.261]",94.0,3.529725,-0.089415,-2.358315
"(-1.261, 0.193]",460.0,4.481341,-0.013768,-2.914280
"(0.193, 1.648]",405.0,3.273574,-0.001622,-3.000512
"(1.648, 3.103]",41.0,1.794934,-0.005213,-1.454618


In [30]:
grouping = pd.qcut(frame.data1,10,labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,3.529725,-0.051769,-2.358315
1,100.0,4.481341,-0.041543,-2.914280
2,100.0,3.230047,0.024076,-2.285779
3,100.0,2.808099,0.026583,-2.467252
4,100.0,3.612836,-0.023807,-2.513474
5,100.0,2.227145,-0.034723,-2.345986
6,100.0,3.273574,-0.077283,-3.000512
7,100.0,3.144789,0.182678,-2.052705
8,100.0,2.418584,-0.065986,-2.484556


# 示例：用特定于分组的值填充缺失值

In [31]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -1.978022
2         NaN
3    0.562497
4         NaN
5   -0.008215
dtype: float64

In [32]:
s.fillna(s.mean())

0   -0.474580
1   -1.978022
2   -0.474580
3    0.562497
4   -0.474580
5   -0.008215
dtype: float64

In [33]:
states = ['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8),index=states)
data

Ohio          0.074712
New York     -0.291495
Vermont      -0.570588
Florida       0.106073
Oregon       -0.473414
Nevada        1.194230
California    1.573380
Idaho         0.704344
dtype: float64

In [34]:
data[['Vermont','Nevada','Idaho']] = np.nan
data

Ohio          0.074712
New York     -0.291495
Vermont            NaN
Florida       0.106073
Oregon       -0.473414
Nevada             NaN
California    1.573380
Idaho              NaN
dtype: float64

In [35]:
data.groupby(group_key).mean()

East   -0.036903
West    0.549983
dtype: float64

In [36]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.074712
New York     -0.291495
Vermont      -0.036903
Florida       0.106073
Oregon       -0.473414
Nevada        0.549983
California    1.573380
Idaho         0.549983
dtype: float64

In [38]:
fill_values = {'East':0.5,'West':-1}
fill_func = lambda g:g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.074712
New York     -0.291495
Vermont       0.500000
Florida       0.106073
Oregon       -0.473414
Nevada       -1.000000
California    1.573380
Idaho        -1.000000
dtype: float64

# 示例：随机采样和排列

In [40]:
suits = ['H','S','C','D']
card_val = (list(range(1,11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2,11)) + ['J','K','Q']
cards = []
for suit in ['H','S','C','D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val,index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [41]:
def draw(dexk,n=5):
    return deck.sample(n)
draw(deck)

4C      4
QS     10
9D      9
10C    10
6C      6
dtype: int64

In [42]:
get_suit = lambda card:card[-1]
deck.groupby(get_suit).apply(draw,n=2)

C  3C     3
   4D     4
D  8S     8
   8C     8
H  5H     5
   8S     8
S  KD    10
   AD     1
dtype: int64

In [51]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

AH      1
8D      8
2S      2
6C      6
10D    10
6H      6
7C      7
JC     10
dtype: int64

# 示例：分组加权平均数和相关系数

In [52]:
df = pd.DataFrame({'category':['a','a','a','a','b','b','b','b'],
                  'data':np.random.randn(8),
                  'weights':np.random.randn(8)})
df

,category,data,weights
0,a,-1.192074,-0.657776
1,a,-1.081399,1.358965
2,a,-0.243915,0.958484
3,a,0.369759,-0.113221
4,b,0.175600,-0.739893
5,b,-1.043637,2.062501
6,b,0.260112,-1.028736
7,b,-0.259184,-0.072104


In [53]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'],weights=g['weights'])
grouped.apply(get_wavg)

category
a    -0.621500
b   -11.414311
dtype: float64

In [54]:
close_px = pd.read_csv('C:\\Mine\\Github\\Python\\stock_px_2.csv',parse_dates=True,index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [55]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [58]:
spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()
get_year = lambda x:x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [59]:
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

# 示例：组级别的线性回归

In [63]:
import statsmodels.api as sm
def regress(data,yvar,xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y,X).fit()
    return result.params
by_year.apply(regress,'AAPL',['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 透视表和交叉表

In [64]:
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [67]:
tips.pivot_table(['tip_pct','size'],
                index=['time','day'],
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [68]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],
                columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [72]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',
                aggfunc=len,margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [76]:
tips.pivot_table('tip_pct',index=['time','size','smoker'],
                columns='day',aggfunc='mean',fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706